In [1]:
from dotenv import load_dotenv
import os, firebase_admin
from firebase_admin import credentials, firestore

# .env 로드
load_dotenv(dotenv_path=".env")

# 이제 환경변수로 인증
key_path = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
cred = credentials.Certificate(key_path)
firebase_admin.initialize_app(cred)
db = firestore.client()

In [2]:
# 2. 과거 이벤트 한 번에 읽어오는 함수
def fetch_events(user_id: str):
    col = db.collection("users").document(user_id).collection("context_events")
    docs = col.order_by("recordTime").stream()
    return [d.to_dict() for d in docs]


# 3. 실시간 추가 감지 콜백
def on_snapshot(col_snapshot, changes, read_time):
    for change in changes:
        if change.type.name == "ADDED":
            new_event = change.document.to_dict()
            print("📥 New event:", new_event)
            # TODO: 여기에서 LLM/Agent 파이프라인 호출


# 4. 리스너 등록 및 초기 데이터 로드
user_id = "test_user_01"
query = db.collection("users").document(user_id).collection("context_events")

# 실시간 리스너
query.on_snapshot(on_snapshot)

# 초기 한 번 로드
events = fetch_events(user_id)
print("🔍 Existing events:", events)

🔍 Existing events: [{'contextData': {'location': {'longitude': -122.083922, 'timestamp': DatetimeWithNanoseconds(2025, 6, 23, 13, 15, 54, 369000, tzinfo=datetime.timezone.utc), 'latitude': 37.4220936}, 'upcomingEvents': []}, 'userId': 'test_user_01', 'recordTime': DatetimeWithNanoseconds(2025, 6, 23, 22, 15, 54, 788970, tzinfo=datetime.timezone.utc)}, {'contextData': {'location': {'longitude': 126.9922937, 'timestamp': DatetimeWithNanoseconds(2025, 6, 25, 11, 43, 36, 53000, tzinfo=datetime.timezone.utc), 'latitude': 37.4924093}, 'upcomingEvents': []}, 'userId': 'test_user_01', 'recordTime': DatetimeWithNanoseconds(2025, 6, 25, 20, 43, 36, 142347, tzinfo=datetime.timezone.utc)}, {'contextData': {'location': {'longitude': 126.9922814, 'timestamp': DatetimeWithNanoseconds(2025, 6, 25, 11, 44, 24, 17000, tzinfo=datetime.timezone.utc), 'latitude': 37.4928651}, 'upcomingEvents': []}, 'userId': 'test_user_01', 'recordTime': DatetimeWithNanoseconds(2025, 6, 25, 20, 44, 24, 73500, tzinfo=dateti

📥 New event: {'contextData': {'location': {'longitude': 126.9976395, 'timestamp': DatetimeWithNanoseconds(2025, 6, 25, 11, 50, 0, 262000, tzinfo=datetime.timezone.utc), 'latitude': 37.4933091}, 'upcomingEvents': []}, 'userId': 'test_user_01', 'recordTime': DatetimeWithNanoseconds(2025, 6, 25, 20, 50, 0, 332727, tzinfo=datetime.timezone.utc)}
📥 New event: {'contextData': {'location': {'longitude': 126.9973728, 'timestamp': DatetimeWithNanoseconds(2025, 6, 25, 11, 51, 7, 497000, tzinfo=datetime.timezone.utc), 'latitude': 37.4927811}, 'upcomingEvents': []}, 'userId': 'test_user_01', 'recordTime': DatetimeWithNanoseconds(2025, 6, 25, 20, 51, 7, 551209, tzinfo=datetime.timezone.utc)}
📥 New event: {'contextData': {'location': {'longitude': 126.9939564, 'timestamp': DatetimeWithNanoseconds(2025, 6, 25, 11, 54, 54, 166000, tzinfo=datetime.timezone.utc), 'latitude': 37.4920709}, 'upcomingEvents': []}, 'userId': 'test_user_01', 'recordTime': DatetimeWithNanoseconds(2025, 6, 25, 20, 54, 54, 32297